# Deep Residual Neural Network

当大家还在惊叹GoogLeNet用结构化的连接纳入了大量卷积层的时候，微软亚洲研究院的研究员已经在设计更深但结构更简单的网络ResNet。他们凭借这个网络在2015年的Imagenet竞赛中大获全胜。

ResNet有效解决了深度卷积神经网络难训练的问题。这是因为在误差反传的过程中，梯度通常变的越来越小，从而权重的更新量也变小。这个导致原理损失函数的层训练缓慢，随着层数的增加这个现象更加明显，之前有两种常用方案来尝试解决这个问题：

* 按层训练。先训练靠近数据的层，然后慢慢的增加后面的层。但效果不是特别好，而且比较麻烦。

* 使用更宽的层（增加输出通道）而不是更深来增加模型复杂度。但更宽的模型经常不如更深的效果好。

ResNet通过增加跨层的连接来解决梯度逐层回传时变小的问题。虽然这个想法之前就提出过了，但ResNet真正的把效果做好了。

下面演示了一个跨层的连接：

<img src="http://zh.gluon.ai/_images/residual.svg">

最底下那层的输入不仅仅是输出给了中间层，而且其与中间层结果相加进入最上层。这样在梯度反传时，最上层梯度可以直接跳过中间层传到最下层，从而避免最下层梯度过小情况。

为什么叫做残差网络呢？我们可以将上面示意图里的结构拆成两个网络的和，一个一层，一个两层，最下面层是共享的。

<img src="http://zh.gluon.ai/_images/residual2.svg">

在训练过程中，左边的网络因为更简单所以更容易训练。这个小网络没有拟合到的部分，或者说残差，则被右边的网络抓取住。所以直观上来说，即使加深网络，跨层连接仍然可以使得底层网络可以充分的训练，从而不会让训练更难。

## ResNet

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-17.png" width="700">

我们可以看到ResNet使用了很多残差块作为其网络的基本结构，并且可以观察到，有的相邻的残差块之间需要通道数量的增加，即增加为原来的2倍，并且随之输出变为原来的1/2，这也是为了考虑每一层中计算的时间复杂度，因此，在设计残差块的时候，我们要考虑到这种通道数和数据宽度的变化。下面我们来设计一个这样的残差块。

In [1]:
import mxnet as mx

from mxnet import nd
from mxnet import gluon
from mxnet import autograd

import sys
sys.path.append('..')
import utils

ctx = mx.gpu()

In [2]:
class Residual(gluon.Block):
    def __init__(self, channels, shape_inc=False, **kwargs):
        super().__init__(**kwargs)
        self.shape_inc = shape_inc
        # 如果channel数增加，我们就要将feature map宽度减半来保持时间复杂度
        strides = 2 if shape_inc else 1
        self.conv1 = gluon.nn.Conv2D(channels, kernel_size=3, strides=strides, padding=1)
        self.bn1 = gluon.nn.BatchNorm(axis=1)
        self.conv2 = gluon.nn.Conv2D(channels, kernel_size=3, strides=1, padding=1)
        self.bn2 = gluon.nn.BatchNorm(axis=1)
        if shape_inc: # 如果channel数增加，则要将feature map宽度减半
            self.conv3 = gluon.nn.Conv2D(channels, kernel_size=1, strides=strides)
            
    def forward(self, X):
        out = nd.relu(self.bn1(self.conv1(X)))
        out = self.bn2(self.conv2(out))
        if self.shape_inc:
            X = self.conv3(X)
        return nd.relu(out + X)

In [3]:
# test
residual = Residual(256, shape_inc=True)
residual.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32, 128, 28, 28), ctx=ctx)
residual(X).shape

(32, 256, 14, 14)

定义好了残差块以后，下面我们实现ResNet18。

In [4]:
class ResNet18(gluon.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super().__init__(**kwargs)
        self.verbose = verbose 
        with self.name_scope():
            b1 = gluon.nn.Sequential()
            b1.add(
                gluon.nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
                gluon.nn.MaxPool2D(pool_size=3, strides=2, padding=1)
            )
            b2 = gluon.nn.Sequential()
            b2.add(
                Residual(64),
                Residual(64)
            )
            b3 = gluon.nn.Sequential()
            b3.add(
                Residual(128, shape_inc=True),
                Residual(128)
            )
            b4 = gluon.nn.Sequential()
            b4.add(
                Residual(256, shape_inc=True),
                Residual(256)
            )
            b5 = gluon.nn.Sequential()
            b5.add(
                Residual(512, shape_inc=True),
                Residual(512)
            )
            b6 = gluon.nn.Sequential()
            b6.add(
                gluon.nn.AvgPool2D(pool_size=7),
                gluon.nn.Flatten(),
                gluon.nn.Dense(num_classes)
            )
            
        self.net = gluon.nn.Sequential()
        self.net.add(b1, b2, b3, b4, b5, b6)
        
        
    def forward(self, X):
        out = X
        for i, blk in enumerate(self.net):
            out = blk(out)
            if self.verbose:
                print("blk %d : %s" % ((i+1), out.shape))
        return out

In [5]:
resnet18 = ResNet18(10, verbose=True)
resnet18.initialize(ctx=ctx)
img = nd.random.normal(shape=(32, 3, 224, 224), ctx=ctx)
y = resnet18(img)

blk 1 : (32, 64, 56, 56)
blk 2 : (32, 64, 56, 56)
blk 3 : (32, 128, 28, 28)
blk 4 : (32, 256, 14, 14)
blk 5 : (32, 512, 7, 7)
blk 6 : (32, 10)


In [6]:
from time import time

batch_size = 32
train_data, test_data = utils.load_dataset(batch_size, resize=299, data_type='cifar10')

resnet18 = ResNet18(10, verbose=False)
resnet18.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

learning_rate = 0.1
trainer = gluon.Trainer(resnet18.collect_params(), 'sgd', {'learning_rate' : learning_rate})

epochs = 20

niter = 0
moving_loss = 0.0
smoothing_constant = 0.9

from time import time
for epoch in range(epochs):
    start = time()
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = resnet18(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        niter += 1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = smoothing_constant * moving_loss + (1-smoothing_constant) * curr_loss
        estimated_loss = moving_loss / (1 - smoothing_constant**niter)
    
    if not (epoch+1)%5:
        learning_rate /= 10
    
    train_acc = utils.evaluate_accuracy_gluon(train_data, resnet18, ctx)
    test_acc = utils.evaluate_accuracy_gluon(test_data, resnet18, ctx)
    print("Epoch %d, Moving Train Avg loss %.5f, Train acc %.5f, Test acc %.5f, Time consume %.5f s."
         % (epoch, estimated_loss, train_acc, test_acc, time() - start))

Epoch 0, Moving Train Avg loss 0.99859, Train acc 0.68838, Test acc 0.66480, Time consume 371.91644 s.
Epoch 1, Moving Train Avg loss 0.57993, Train acc 0.82110, Test acc 0.78250, Time consume 371.53594 s.
Epoch 2, Moving Train Avg loss 0.46610, Train acc 0.86808, Test acc 0.80640, Time consume 370.99050 s.
Epoch 3, Moving Train Avg loss 0.36022, Train acc 0.90014, Test acc 0.80880, Time consume 371.29495 s.
Epoch 4, Moving Train Avg loss 0.34219, Train acc 0.93746, Test acc 0.82690, Time consume 371.36973 s.
Epoch 5, Moving Train Avg loss 0.23900, Train acc 0.93820, Test acc 0.81380, Time consume 371.33454 s.
Epoch 6, Moving Train Avg loss 0.14782, Train acc 0.96718, Test acc 0.82470, Time consume 371.36895 s.
Epoch 7, Moving Train Avg loss 0.11006, Train acc 0.97052, Test acc 0.82690, Time consume 371.24905 s.
Epoch 8, Moving Train Avg loss 0.08237, Train acc 0.95556, Test acc 0.79880, Time consume 371.20606 s.
Epoch 9, Moving Train Avg loss 0.05432, Train acc 0.99018, Test acc 0.837

ResNet的效果确实好啊！！！